<a href="https://colab.research.google.com/github/gojiteji/NAGISystem/blob/main/thatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
modelname="EleutherAI/gpt-neo-2.7B"
config = AutoConfig.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForCausalLM.from_pretrained(modelname,config=config).to("cuda")

In [145]:
def botsay(user_input):
  prompt = "This is a conversation between Human and AI bot. AI's name is ThatGPT."  
  new_token_id=None
  gen_tokens=""
  new_token=""
  j =6
  length=0
  limit = 128
  thatid=5562
  cont = True
  last_apppended = False
  cnt=0
  disable_repeat_length= 5
  disable_repeat_count = 2
  tokens=[]
  while(cont):
    cnt+=1
    prob = 1.0
    input_ids=tokenizer(prompt+user_input+"\nAI:"+gen_tokens,return_tensors="pt").input_ids
    length=len(input_ids)
    if length >limit:
      gen_tokens="⚠️sorry length limit. please reload the browser."
      return gen_tokens
    outs=model(input_ids=input_ids.to("cuda"))
    topk = torch.topk(outs.logits.squeeze()[-1,:],k=j+1).indices
    if new_token =="that":
      that_id = 326
    elif new_token ==" that":
      that_id = -1
    elif new_token[-1:] ==" ":
      that_id = 5562
    else:
      that_id = 326

    if ("thatGPT" in gen_tokens[-12:]):
      that_id = -1
    if last_apppended:
      that_id = -1
    if that_id in topk:
      new_token_id = that_id
    else:
      new_token_id = torch.argmax(outs.logits.squeeze()[-1,:])
      new_token=tokenizer.decode(new_token_id)
    new_token=tokenizer.decode(new_token_id)
    prev_tokens=gen_tokens
    gen_tokens+=new_token
    if (cnt>10) and (disable_repeat_count<gen_tokens.count(gen_tokens[-disable_repeat_length:])):
      gen_tokens=prev_tokens
      new_token = tokenizer.decode(topk[torch.randint(5, (1,1)).item()])
      gen_tokens+=new_token

    if new_token_id==50256 or new_token_id==198 or new_token=="<|endoftext|>":
      if ("that" not in gen_tokens):
        gen_tokens = gen_tokens.replace("\n","").replace(".","")
        gen_tokens += " that"
      else:
        cont = False
  return gen_tokens.replace("<br>","").replace("AI:","").replace("\xa0","")

In [151]:
import gradio as gr
def add_text(history, text):
    history = history + [(text, None)]
    return history, ""


def bot(history):
    serial_history=""
    for h in history:
      serial_history+="\nHuman:"+h[0]
      if h[1]==None:
        break
      serial_history+="\nAI:"+h[1].replace("<br>","")

    response = botsay(serial_history)
    history[-1][1] = response
    serial_history+="\nAI:"+response
    return history

with gr.Blocks() as demo:
    gr.Markdown("# ThatGPT")
    chatbot = gr.Chatbot([], elem_id="chatbot").style(height=750)

    with gr.Row():
        with gr.Column(scale=0.85):
            txt = gr.Textbox(
                show_label=False,
                placeholder="AI always replies with \"that\".",
            ).style(container=False)
        
    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e01381ce9f1eb85ae1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e01381ce9f1eb85ae1.gradio.live
